In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as sig
import scipy.io.wavfile as wavfile
import random

MASTER_FS = 44100

In [ ]:
fs, x = wavfile.read("music/band_classification/andy_mckee_into_the_ocean.wav")
x.shape

In [ ]:
def read_song(fname):
    fs, x = wavfile.read(fname)
    assert(x.shape[1] == 2)
    assert(fs == MASTER_FS)  # If not true gonna have to resample
    
    return x

def random_five_seconds(song, fs):
    samples = x.shape[0]
    snip_length = fs*5
    index = random.randint(0, samples-snip_length)
    return song[index:index+snip_length]

def five_sec_spectrogram(song, fs):
    sample = random_five_seconds(song, fs)
    f, t, sxx0 = sig.spectrogram(sample[:, 0], fs=fs)
    _, _, sxx1 = sig.spectrogram(sample[:, 1], fs=fs)
    
    return (f, t, sxx0, sxx1)

def svd_of_stacked_spectrograms(sxx0, sxx1):
    # We want time to progress by row
    # So we want to transpose and hstack?
    data_matrix = np.hstack((sxx0.T, sxx1.T))
    u, s, vh = np.linalg.svd(data_matrix, full_matrices=False)
    
    return u, s, vh

In [ ]:
f, t, sxx0, sxx1 = five_sec_spectrogram(x, 44100)
sxx0.T.shape
plt.figure()

In [ ]:
svd_of_stacked_spectrograms(sxx0, sxx1)